# CGMF Summary File Generator

<font size=4em color="darkblue">Austin Carter<sup>a,b</sup>, Patrick Talou<sup>a</sup>, Ionel Stetcu<sup>a</sup></font> <br />
<sup>a</sup>Los Alamos National Laboratory <br />
<sup>b</sup>University of Michigan<br />

<a href="mailto:austinlc@umich.edu">austinlc@umich.edu</a>

<a id="TOC"></a>
## Table of Contents

<ul>
<li><a href="#Introduction">Introduction</a></li>
<li><a href="#Notebook Libraries and Settings">Notebook Libraries and Settings</a></li>
<li><a href="#Define CGMF Output Directory">Define CGMF Output Directory</a></li>
<li><a href="#Define and Initialize Summary File">Define and Initialize Summary File</a></li>
<li><a href="#Gather Data and Print to Summary File">Gather Data and Print to Summary File</a></li>
</ul>

<a id="Introduction"></a><h2>Introduction</h2>

The purpose of this notebook is to allow the CGMF analyst to organize data from numerous CGMF output files into a summary file. The output files are read individually. The summary file contains $$$. 

<a id="Notebook Libraries and Settings"></a><h2>Notebook Libraries and Settings</h2>

In [9]:
import numpy as np
import scipy as sp
import os
from statistics import *
from datetime import datetime

<a id="Define CGMF Output Directory"></a><h2>Define CGMF Output Directory</h2>

In [10]:
# First Specify the Output files to be read
# ALl output files must be in a subdirectory of the current working directory called "OutputFiles"

filepath = os.path.join(os.getcwd(),'OutputFiles')
if not os.path.isdir(filepath):
    raise ValueError('You do not have the directory "OutputFiles" in your current working directory')

print(os.listdir(filepath))

['output23.txt', 'output1.txt', 'output4.txt', 'output16.txt', 'output26.txt', 'output56.txt', 'output42.txt', 'output40.txt', 'output43.txt', 'output55.txt', 'output44.txt', 'output60.txt', 'output15.txt', 'output8.txt', 'output39.txt', 'output29.txt', 'output53.txt', 'output34.txt', 'output24.txt', 'output63.txt', 'output3.txt', 'output14.txt', 'output49.txt', 'output33.txt', 'output37.txt', 'output46.txt', 'output54.txt', 'output59.txt', 'output25.txt', 'output12.txt', 'output21.txt', 'output51.txt', 'output20.txt', 'output5.txt', 'output22.txt', 'output27.txt', 'output30.txt', 'output11.txt', 'output9.txt', 'output52.txt', 'output50.txt', 'output58.txt', 'output10.txt', 'output2.txt', 'output32.txt', 'output45.txt', 'output48.txt', 'output57.txt', 'output36.txt', 'output6.txt', 'output28.txt', 'output62.txt', 'output47.txt', 'output13.txt', 'output17.txt', 'output35.txt', 'output61.txt', 'output19.txt', 'output18.txt', 'output38.txt', 'output31.txt', 'output41.txt', 'output64.txt',

<a id="Define and Initialize Summary File"></a><h2>Define and Initialize Summary File</h2>

In [11]:
# Define summary file path and modify if necessary
summary_file_path = os.path.join(os.getcwd(),"Summary.txt")
if os.path.isfile(summary_file_path):
    #ALWAYS OVERWRITES
#     overwrite = True
#     if (overwrite == "Yes" or overwrite =="yes" or overwrite =='y' or overwrite =='Y'):
        f = open(summary_file_path, 'w')
        f.close()
#     elif(overwrite == "No" or overwrite =="no" or overwrite =='n' or overwrite =='N'):
#         summary_file_name = input("Please Specify Summary File Name (with extension): ")
#         summary_file_path = os.path.join(os.getcwd(),summary_file_name)
        
# Add headings to Summary File
f = open(summary_file_path, 'w')
f.write("CGMF SUMMARY FILE\n")
f.write("Generated: " + str(datetime.now().strftime("%Y-%m-%d %H:%M")) + "\n")
f.write("Number of CGMF Output Files: {0:2d}\n\n".format(len(os.listdir(filepath))))
f.write("File\t\t# Hist\t<TXE>\t1sig\t\t<TKE>\t1sig\t\t<J>\t1sig\t\t<nn>\tn-mom2\tn-mom3\t<ng>\tg-mom2\tg-mom3\t<En>\t1sig\t\t<Eg>\t1sig\n")
f.write("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")
f.close()

<a id="Gather Data and Print to Summary File"></a><h2>Gather Data and Print to Summary File</h2>

In [12]:
# if user desires, a specific number of histories can be read, otherwise the max is infinite
MaxHistories = 10000

# total number of files to be read
n_files = len(os.listdir(filepath))

# initialize the index of the files to 0
file_index = 0

# From filepath, read the files individually
# reads all files in order according to listdir

for filename in os.listdir(filepath):
    
    file_index += 1
    file_loc = os.path.join(filepath,filename)
    working_file = open(file_loc)
    
    # read first line -----------------------------------------------------------------------------------------------
    line = working_file.readline()
    if (line.find("#")!=0):
        sys.exit("ERROR: FIRST LINE OF OUTPUT FILE SHOULD CONTAIN '#'")
    else:    
        data          = line[1:].split()
        ZAIDc         = int(data[0])
        Zc            = int(ZAIDc/1000)
        Ac            = int(ZAIDc)-1000*Zc
        Asym          = int(float(Ac)/2.0)
        Einc          = float(data[1])
        nevents       = int(data[2])
        alpha         = float(data[3])
        
    # read remainder -----------------------------------------------------------------------------------------------
    nfragments = 0
    
    # re-initialize all arrays
    # Line 1 Arrays of interest

    Ul, Uh, TXE      = [], [], []
    Jl, Jh, J        = [], [], []
    KEl, KEh, TKE    = [], [], []
    nnl, nnh, nnt    = [], [], []
    ngl, ngh, ngt    = [], [], []

    # Direction/momentum vectors are arranged with CoM first, then Lab
    # Fragment momentum arrays - Line 2
    #Pcml, Pcmh       = [], []
    #Plabl, Plabh     = [], []
    
    # For now, just save the resulting particle energies
    # Neutron Energies - Line 3
    Encml, Encmh     = [], []
    Enlabl, Enlabh   = [], []
    
    # Gamma Energies - Line 4
    Egcml, Egcmh     = [], []
    Eglabl, Eglabh   = [], []
    
    while True:
        line = working_file.readline()
        if (len(line)==0):
            break
        #LINE ONE OF FISSION EVENT (A, Z, U, J, P, KE, nn, ng, nIC)
        nfragments+=1
        tempdata1 = line.split()
        if (nfragments % 2 == 1): # LIGHT FRAGMENT
            Ul.append(float(tempdata1[2]))
            Jl.append(float(tempdata1[3]))
            KEl.append(float(tempdata1[5]))
            nnl.append(int(tempdata1[6]))
            ngl.append(int(tempdata1[7]))

        else: # HEAVY FRAGMENT
            Uh.append(float(tempdata1[2]))
            Jh.append(float(tempdata1[3]))
            KEh.append(float(tempdata1[5]))
            nnh.append(int(tempdata1[6]))
            ngh.append(int(tempdata1[7]))

        
        #LINE TWO OF FISSION EVENT (momentum vector in Lab and center of mass of FRAGMENT)
        line = working_file.readline()
        tempdata2 = line.split()
        # for now, don't worry about line 2
        
        #LINE THREE OF FISSION EVENT (Neutron direction cosines and energies, Lab and center of mass respectively)
        if (int(tempdata1[6]) != 0):
            line = working_file.readline()
            tempdata3 = line.split()
            if (nfragments % 2 == 1): # LIGHT FRAGMENT
                for i in range(int(tempdata1[6])):
                    Encml.append(float(tempdata3[4*i + 7]))
                    Enlabl.append(float(tempdata3[8*i + 7]))
            else: # HEAVY FRAGMENT
                for i in range(int(tempdata1[6])):
                    Encmh.append(float(tempdata3[4*i + 7]))
                    Enlabh.append(float(tempdata3[8*i + 7]))
            
        
        #LINE FOUR OF FISSION EVENT (Gamma direction cosines and energies, Lab and center of mass respectively)
        if (int(tempdata1[7]) != 0):
            line = working_file.readline()
            tempdata4 = line.split()
            if (nfragments % 2 == 1): # LIGHT FRAGMENT
                for i in range(int(tempdata1[7])):
                    Egcml.append(float(tempdata4[4*i + 7]))
                    Eglabl.append(float(tempdata4[8*i + 7]))
            else: # HEAVY FRAGMENT
                for i in range(int(tempdata1[7])):
                    Egcmh.append(float(tempdata4[4*i + 7]))
                    Eglabh.append(float(tempdata4[8*i + 7]))
        
        if (nfragments>=MaxHistories*2):
            break
    nhistories = nfragments/2
    
    # Concatenate arrays
    TXE    = np.sum([Ul , Uh], axis=0)
    J      = Jl + Jh
    TKE    = np.sum([KEl , KEh], axis=0)
    nnt    = np.sum([nnl , nnh], axis=0)
    nn2t   = [float(nnt[i] * (nnt[i]-1)) for i in range(len(nnt))] # 2nd factorial moment
    nn3t   = [float(nnt[i] * (nnt[i]-1) * (nnt[i]-2)) for i in range(len(nnt))] # 3rd factorial moment
    ngt    = np.sum([ngl , ngh], axis=0)
    ng2t   = [float(ngt[i] * (ngt[i]-1)) for i in range(len(ngt))]
    ng3t   = [float(ngt[i] * (ngt[i]-1) * (ngt[i]-2)) for i in range(len(ngt))]
    Encm  = Encml + Encmh
    Egcm  = Egcml + Egcmh
    
    # Close File that is read
    working_file.close()
    
    # Open Summary File
    
    summary = open(summary_file_path,"a")
    concstr = filename + "\t{:d}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\t\
{:0.3f}\t{:0.3f}\t{:0.3f}\t{:0.3f}\t{:0.3f}\t{:0.2f}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\n".format(
                    int(nhistories),
                    mean(TXE), pstdev(TXE),  
                    mean(TKE), pstdev(TKE), mean(J),
                    pstdev(J), mean(nnt.astype(float)), mean(nn2t),
                    mean(nn3t), mean(ngt.astype(float)), mean(ng2t), mean(ng3t), mean(Encm), pstdev(Encm), 
                    mean(Egcm), np.sqrt(pstdev(Egcm)/nhistories))

    summary.write(concstr)
    summary.close()
    print( "{:0.1f}% complete".format(file_index/n_files*100))

1.6% complete
3.1% complete
4.7% complete
6.2% complete
7.8% complete
9.4% complete
10.9% complete
12.5% complete
14.1% complete
15.6% complete
17.2% complete
18.8% complete
20.3% complete
21.9% complete
23.4% complete
25.0% complete
26.6% complete
28.1% complete
29.7% complete
31.2% complete
32.8% complete
34.4% complete
35.9% complete
37.5% complete
39.1% complete
40.6% complete
42.2% complete
43.8% complete
45.3% complete
46.9% complete
48.4% complete
50.0% complete
51.6% complete
53.1% complete
54.7% complete
56.2% complete
57.8% complete
59.4% complete
60.9% complete
62.5% complete
64.1% complete
65.6% complete
67.2% complete
68.8% complete
70.3% complete
71.9% complete
73.4% complete
75.0% complete
76.6% complete
78.1% complete
79.7% complete
81.2% complete
82.8% complete
84.4% complete
85.9% complete
87.5% complete
89.1% complete
90.6% complete
92.2% complete
93.8% complete
95.3% complete
96.9% complete
98.4% complete
100.0% complete
